In [ ]:
import mmcv
from mmaction.apis import init_recognizer, inference_recognizer
import numpy as np
import os, glob
import json
from collections import defaultdict

# Define available models for comparison
models = {
    # SlowFast (keep one or two)
    'slowfast_r50_8x8x1': {
        'config': 'configs\\recognition\\slowfast\\slowfast_r50_8xb8-8x8x1-steplr-256e_kinetics400-rgb.py',
        'checkpoint': 'checkpoints\\slowfast\\slowfast_r50_8xb8-8x8x1-steplr-256e_kinetics400-rgb_20220818-b62a501f.pth',
        'labels': 'tools/data/kinetics/label_map_k400.txt'
    },

    # TSN (lightweight baseline)
    # Pick an r50 1x1x8 K400 config that exists in your repo; adjust the two paths below.
    'tsn_r50_1x1x8': {
        'config': 'configs\\recognition\\tsn\\tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb.py',
        'checkpoint': 'checkpoints\\tsn\\tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth',
        'labels': 'tools/data/kinetics/label_map_k400.txt'
    },

    # TSM (better temporal modeling at TSN-like cost)
    'tsm_r50_1x1x8': {
        'config': 'configs\\recognition\\tsm\\tsm_imagenet-pretrained-r50_8xb16-1x1x8-50e_kinetics400-rgb.py',
        'checkpoint': 'checkpoints\\tsm\\tsm_imagenet-pretrained-r50_8xb16-1x1x8-50e_kinetics400-rgb_20220831-64d69186.pth',
        'labels': 'tools/data/kinetics/label_map_k400.txt'
    }
}

# Discover videos from folders
def list_videos(folder):
    exts = ('*.mp4', '*.avi', '*.mov', '*.mkv')
    files = []
    for ext in exts:
        files += glob.glob(os.path.join(folder, ext))
    return sorted(files)

impact_dir = r'inputs\videos\impact'
normal_dir = r'inputs\videos\normal'

video_categories = {
    'impact': list_videos(impact_dir),
    'normal': list_videos(normal_dir)
}

print("Discovered videos:")
print(f"- impact: {len(video_categories['impact'])} files")
print(f"- normal: {len(video_categories['normal'])} files")
if video_categories['impact'][:3]:
    print("  sample impact:", [os.path.basename(p) for p in video_categories['impact'][:3]])
if video_categories['normal'][:3]:
    print("  sample normal:", [os.path.basename(p) for p in video_categories['normal'][:3]])


# Define collision-related labels to analyze
collision_groups = {
    'primary': [
        'tackling', 'headbutting', 'wrestling',
        'punching person (boxing)', 'slapping', 'side kick', 'drop kicking',
        'sword fighting'
    ],
    'context': [
        'playing rugby', 'playing ice hockey', 'hurling (sport)',
        'passing American football (in game)', 'passing American football (not in game)'
    ],
    'aftermath': [
        'faceplanting'
    ]
}

collision_weights = {
    'primary': 1.0,
    'aftermath': 0.7,
    'context': 0.4
}

Discovered videos:
- impact: 9 files
- normal: 9 files
  sample impact: ['Recording 2025-10-22 135626.mp4', 'Recording 2025-10-22 141700.mp4', 'player_1307_full_1.8s.mp4']
  sample normal: ['player_1121_full_8.3s.mp4', 'player_171_full_7.8s.mp4', 'player_240_full_5.8s.mp4']


In [2]:
def load_labels(label_file):
    """Load label mapping from file"""
    with open(label_file, 'r') as f:
        labels = [line.strip() for line in f.readlines()]
    return labels

def analyze_video(model, video_path, labels):
    """Analyze a single video and return predictions"""
    try:
        result = inference_recognizer(model, video_path)
        pred_scores = result.pred_score.cpu().numpy()
        
        # Get top 10 predictions for analysis
        top_indices = np.argsort(pred_scores)[::-1][:10]
        
        predictions = []
        for idx in top_indices:
            predictions.append({
                'label': labels[idx],
                'score': float(pred_scores[idx]),
                'index': int(idx)
            })
        
        return predictions
    
    except Exception as e:
        print(f"Error analyzing {video_path}: {str(e)}")
        return None

def calculate_collision_score(predictions, groups=collision_groups, weights=collision_weights):
    label_to_weight = {}
    for k, lst in groups.items():
        for L in lst:
            label_to_weight[L] = weights.get(k, 0.0)

    score = 0.0
    hits = []
    for pred in predictions:
        w = label_to_weight.get(pred['label'], 0.0)
        if w > 0:
            score += w * pred['score']
            hits.append((pred['label'], pred['score'], w))
    return score, hits

In [3]:
# Run the analysis
print("🔍 AFL COLLISION DETECTION ANALYSIS")
print("=" * 60)

results = defaultdict(lambda: defaultdict(list))
detailed_results = []

# Analyze each model
for model_name, model_info in models.items():
    print(f"\n📊 Testing {model_name.upper()}")
    print("-" * 40)
    
    # Load model
    try:
        model = init_recognizer(model_info['config'], model_info['checkpoint'], device='cuda:0')
        labels = load_labels(model_info['labels'])
        print(f"✅ Model loaded successfully")
    except Exception as e:
        print(f"❌ Failed to load {model_name}: {str(e)}")
        continue
    
    # Analyze each video category
    for category, video_list in video_categories.items():
        if not video_list:  # Skip empty categories
            print(f"  ⚠️  No {category} videos provided")
            continue
            
        print(f"\n  📹 Analyzing {category.upper()} videos:")
        
        for video_path in video_list:
            if not os.path.exists(video_path):
                print(f"    ⚠️  Video not found: {video_path}")
                continue
            
            print(f"    🎬 Processing: {os.path.basename(video_path)}")
            
            # Analyze video
            predictions = analyze_video(model, video_path, labels)
            if predictions is None:
                continue
            
            # Calculate collision score
            collision_score, collision_labels = calculate_collision_score(predictions, collision_groups, collision_weights)
            
            # Store results
            result_entry = {
                'model': model_name,
                'category': category,
                'video': video_path,
                'top_prediction': predictions[0],
                'collision_score': collision_score,
                'collision_labels': collision_labels,
                'all_predictions': predictions[:5]  # Top 5
            }
            
            results[model_name][category].append(result_entry)
            detailed_results.append(result_entry)
            
            # Display key results
            print(f"      🏆 Top prediction: {predictions[0]['label']} ({predictions[0]['score']:.4f})")
            print(f"      ⚔️  Collision score: {collision_score:.4f}")
            if collision_labels:
                print(f"      🎯 Collision indicators: {collision_labels[:2]}")  # Show top 2

# Generate Analysis Report
print("\n" + "=" * 60)
print("📈 ANALYSIS RESULTS")
print("=" * 60)

# Model Performance Summary
print("\n1️⃣ MODEL PERFORMANCE SUMMARY:")
print("-" * 30)

for model_name in results.keys():
    print(f"\n🤖 {model_name.upper()}:")
    
    # Calculate average collision scores
    impact_scores = []
    normal_scores = []
    
    for entry in detailed_results:
        if entry['model'] == model_name:
            if entry['category'] == 'impact':
                impact_scores.append(entry['collision_score'])
            else:
                normal_scores.append(entry['collision_score'])
    
    avg_impact = np.mean(impact_scores) if impact_scores else 0
    avg_normal = np.mean(normal_scores) if normal_scores else 0
    separation = avg_impact - avg_normal
    
    print(f"  📊 Avg collision score - Impact: {avg_impact:.4f}, Normal: {avg_normal:.4f}")
    print(f"  🎯 Separation ability: {separation:.4f}")

# Collision Detection Capability
print("\n2️⃣ COLLISION DETECTION CAPABILITY:")
print("-" * 30)

all_impact_scores = [entry['collision_score'] for entry in detailed_results if entry['category'] == 'impact']
all_normal_scores = [entry['collision_score'] for entry in detailed_results if entry['category'] == 'normal']

if all_impact_scores and all_normal_scores:
    threshold_candidates = np.linspace(0, max(all_impact_scores + all_normal_scores), 20)
    best_threshold = 0
    best_accuracy = 0
    
    for threshold in threshold_candidates:
        tp = sum(1 for score in all_impact_scores if score >= threshold)
        tn = sum(1 for score in all_normal_scores if score < threshold)
        total = len(all_impact_scores) + len(all_normal_scores)
        accuracy = (tp + tn) / total
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold
    
    print(f"🎯 Optimal collision threshold: {best_threshold:.4f}")
    print(f"📊 Achievable accuracy: {best_accuracy:.2%}")
    
    # Precision and Recall
    tp = sum(1 for score in all_impact_scores if score >= best_threshold)
    fp = sum(1 for score in all_normal_scores if score >= best_threshold)
    fn = sum(1 for score in all_impact_scores if score < best_threshold)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    print(f"🎯 Precision: {precision:.2%}")
    print(f"🎯 Recall: {recall:.2%}")
else:
    print("⚠️  Need both impact and normal videos for threshold analysis")

# Research Question Answer
print("\n3️⃣ RESEARCH QUESTION ANSWER:")
print("-" * 30)
print("❓ How can existing AI frameworks be adapted for AFL collision detection?")
print()

if 'best_accuracy' in locals() and best_accuracy > 0.7:
    print("✅ PROMISING APPROACH:")
    print("   • Existing models show good separation")
    print(f"   • Simple thresholding achieves {best_accuracy:.1%} accuracy")
    print("   • Recommend: Fine-tune with AFL-specific dataset")
elif 'best_accuracy' in locals() and best_accuracy > 0.6:
    print("⚠️  MODERATE POTENTIAL:")
    print("   • Some separation exists but needs improvement")
    print("   • Recommend: Transfer learning with binary classification")
else:
    print("🔍 PRELIMINARY RESULTS:")
    print("   • Need more diverse video samples for full evaluation")
    print("   • Current models show varying collision detection capability")
    print("   • Recommend: Expand dataset and test transfer learning")

# Recommendations
print("\n4️⃣ NEXT STEPS:")
print("-" * 30)
print("🎯 IMMEDIATE ACTIONS:")
print("   1. Collect more labeled AFL collision videos")
print("   2. Implement binary classifier (impact/normal)")
print("   3. Use transfer learning from best-performing model")

print(f"\n✅ Analysis complete! Processed {len(detailed_results)} video-model combinations.")

🔍 AFL COLLISION DETECTION ANALYSIS

📊 Testing SLOWFAST_R50_8X8X1
----------------------------------------
Loads checkpoint by local backend from path: checkpoints\slowfast\slowfast_r50_8xb8-8x8x1-steplr-256e_kinetics400-rgb_20220818-b62a501f.pth
✅ Model loaded successfully

  📹 Analyzing IMPACT videos:
    🎬 Processing: Recording 2025-10-22 135626.mp4
10/22 16:50:56 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
10/22 16:50:56 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
      🏆 Top prediction: hurling (sport) (0.9852)
      ⚔️  Collision score: 0.3966
      🎯 Collision indicators: [('hurling (sport)', 0.9851663708686829, 0.4), ('passing American football (in game)', 0.004583391826599836, 0.4)]
    🎬 Processing: Recording 2025-10-22 141700.mp4
      🏆 Top prediction: shooting goal (soccer) (0.4888)
      

In [ ]:
# # Save raw results to JSON/CSV and build a per-video dataframe
# import os, json
# import pandas as pd
# from datetime import datetime

out_dir = "outputs"
os.makedirs(out_dir, exist_ok=True)

# 1) JSON dump
stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
json_path = os.path.join(out_dir, f"detailed_results_{stamp}.json")
with open(json_path, "w") as f:
    json.dump(detailed_results, f, indent=2)
print(f"Saved JSON: {json_path}")

# 2) Per-video CSV (flattened)
rows = []
for r in detailed_results:
    row = {
        "model": r["model"],
        "category": r["category"],
        "video": r["video"],
        "basename": os.path.basename(r["video"]),
        "collision_score": r["collision_score"],
        "top1_label": r["top_prediction"]["label"],
        "top1_score": r["top_prediction"]["score"],
    }
    # top-5 predictions
    for i, p in enumerate(r.get("all_predictions", [])[:5], start=1):
        row[f"top{i}_label"] = p["label"]
        row[f"top{i}_score"] = p["score"]
    # collision hits with weights
    hits = r.get("collision_labels", [])
    row["collision_hits"] = "; ".join([f"{lbl}:{s:.3f}*w{w:.1f}" for (lbl, s, w) in hits])
    rows.append(row)

df = pd.DataFrame(rows)
csv_path = os.path.join(out_dir, f"per_video_results_{stamp}.csv")
df.to_csv(csv_path, index=False)
print(f"Saved CSV: {csv_path}")

# 3) Also save one CSV per model
for m in df["model"].unique():
    m_df = df[df["model"] == m].copy()
    m_csv = os.path.join(out_dir, f"{m}_per_video_{stamp}.csv")
    m_df.to_csv(m_csv, index=False)
    print(f"Saved per-model CSV: {m_csv}")

Saved JSON: eval_outputs\detailed_results_20251022_165416.json
Saved CSV: eval_outputs\per_video_results_20251022_165416.csv
Saved per-model CSV: eval_outputs\slowfast_r50_8x8x1_per_video_20251022_165416.csv
Saved per-model CSV: eval_outputs\tsn_r50_1x1x8_per_video_20251022_165416.csv
Saved per-model CSV: eval_outputs\tsm_r50_1x1x8_per_video_20251022_165416.csv


In [ ]:

# # SAFE plotting of Top-5 label frequencies across categories and models (no seaborn)
# import os, gc, warnings
# # Make sure env vars are set in case this cell is run standalone
# os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")
# os.environ.setdefault("OMP_NUM_THREADS", "1")
# os.environ.setdefault("MKL_NUM_THREADS", "1")
# os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# import numpy as np
# import pandas as pd
# import matplotlib
# matplotlib.use("Agg")  # non-interactive backend to avoid GUI/memory issues
# import matplotlib.pyplot as plt

# warnings.filterwarnings("ignore")
# plt.ioff()  # disable interactive rendering to reduce RAM usage

# out_dir = globals().get("out_dir", "eval_outputs")
# os.makedirs(out_dir, exist_ok=True)
# stamp = globals().get("stamp", __import__("datetime").datetime.now().strftime("%Y%m%d_%H%M%S"))

# # Build long table of top-5 labels per video
# label_cols = [f"top{i}_label" for i in range(1, 6)]
# assert all(c in df.columns for c in label_cols), "Expected top1_label..top5_label in df"

# long = df.melt(
#     id_vars=["model", "category", "video", "basename"],
#     value_vars=label_cols,
#     var_name="rank",
#     value_name="label",
# ).dropna(subset=["label"])

# # Count occurrences and normalize by number of clips per (model, category)
# counts = (long.groupby(["category", "model", "label"])
#                .size().reset_index(name="count"))
# n_vids = (df.groupby(["model", "category"])
#             .size().reset_index(name="n_videos"))
# counts = counts.merge(n_vids, on=["model","category"], how="left")
# counts["freq_per_clip"] = counts["count"] / counts["n_videos"].clip(lower=1)

# def safe_save(fig, path):
#     try:
#         fig.savefig(path, dpi=160, bbox_inches="tight")  # slightly lower dpi to reduce memory
#         print(f"Saved: {path}")
#     finally:
#         plt.close(fig); gc.collect()

# def plot_category_safe(category, top_n=5):
#     sub = counts[counts["category"] == category].copy()
#     if sub.empty:
#         print(f"No data for category={category}")
#         return

#     # Select global Top-N labels for this category
#     top_labels = (sub.groupby("label")["count"]
#                     .sum().sort_values(ascending=False)
#                     .head(top_n).index.tolist())
#     sub = sub[sub["label"].isin(top_labels)].copy()
#     sub["label"] = pd.Categorical(sub["label"], categories=top_labels, ordered=True)

#     # Grouped bar chart (frequency per clip)
#     models = list(sub["model"].unique())
#     x = np.arange(len(top_labels))
#     width = 0.8 / max(1, len(models))

#     fig, ax = plt.subplots(figsize=(9.5, 4.6))
#     for i, m in enumerate(models):
#         s = (sub[sub["model"] == m]
#                 .set_index("label")["freq_per_clip"]
#                 .reindex(top_labels).fillna(0.0).values)
#         ax.bar(x + i*width - (len(models)-1)*width/2, s, width=width, label=m)
#     ax.set_title(f"Top-{len(top_labels)} label frequency per clip — {category}")
#     ax.set_xticks(x); ax.set_xticklabels(top_labels, rotation=30, ha="right")
#     ax.set_ylabel("frequency per clip"); ax.set_xlabel("label")
#     ax.legend(loc="upper right", fontsize=9)
#     safe_save(fig, os.path.join(out_dir, f"safe_top_labels_{category}_grouped_{stamp}.png"))

#     # Heatmap (absolute counts)
#     pivot_cnt = (sub.pivot_table(index="label", columns="model", values="count", aggfunc="sum")
#                    .reindex(index=top_labels).fillna(0).astype(int))
#     data = pivot_cnt.values
#     fig, ax = plt.subplots(figsize=(8, 4.0))
#     im = ax.imshow(data, aspect="auto", cmap="Blues")
#     ax.set_title(f"Top-{len(top_labels)} label counts — {category}")
#     ax.set_yticks(np.arange(len(top_labels))); ax.set_yticklabels(top_labels)
#     ax.set_xticks(np.arange(len(models))); ax.set_xticklabels(models, rotation=15, ha="right")
#     # annotate cells
#     for i in range(data.shape[0]):
#         for j in range(data.shape[1]):
#             ax.text(j, i, int(data[i, j]), ha="center", va="center", color="black", fontsize=9)
#     fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
#     safe_save(fig, os.path.join(out_dir, f"safe_top_labels_{category}_heatmap_{stamp}.png"))

#     # Export compact CSV tables (counts and normalized freq)
#     cnt_csv = os.path.join(out_dir, f"safe_top5_{category}_counts_{stamp}.csv")
#     frq_csv = os.path.join(out_dir, f"safe_top5_{category}_freq_per_clip_{stamp}.csv")
#     pivot_cnt.to_csv(cnt_csv)
#     sub.pivot_table(index="label", columns="model", values="freq_per_clip", aggfunc="mean").reindex(top_labels).to_csv(frq_csv)
#     print("Saved tables:", cnt_csv, "and", frq_csv)

# plot_category_safe("impact", top_n=5)
# plot_category_safe("normal", top_n=5)

Saved: eval_outputs\safe_top_labels_impact_grouped_20251022_165416.png
Saved: eval_outputs\safe_top_labels_impact_heatmap_20251022_165416.png
Saved tables: eval_outputs\safe_top5_impact_counts_20251022_165416.csv and eval_outputs\safe_top5_impact_freq_per_clip_20251022_165416.csv
Saved: eval_outputs\safe_top_labels_normal_grouped_20251022_165416.png
Saved: eval_outputs\safe_top_labels_normal_heatmap_20251022_165416.png
Saved tables: eval_outputs\safe_top5_normal_counts_20251022_165416.csv and eval_outputs\safe_top5_normal_freq_per_clip_20251022_165416.csv


In [ ]:
# import os, gc, warnings
# import numpy as np
# import pandas as pd
# import matplotlib
# matplotlib.use("Agg")
# import matplotlib.pyplot as plt
# from matplotlib.ticker import MaxNLocator

# warnings.filterwarnings("ignore")
# plt.ioff()

# out_dir = globals().get("out_dir", "eval_outputs")
# os.makedirs(out_dir, exist_ok=True)
# stamp = globals().get("stamp", __import__("datetime").datetime.now().strftime("%Y%m%d_%H%M%S"))

# # Build long table with explicit rank 1..5
# label_cols = [f"top{i}_label" for i in range(1, 6)]
# assert all(c in df.columns for c in label_cols), "Expected top1_label..top5_label in df"

# long = df.melt(
#     id_vars=["model", "category", "video", "basename"],
#     value_vars=label_cols,
#     var_name="rank",
#     value_name="label",
# ).dropna(subset=["label"]).copy()
# long["rank"] = long["rank"].str.extract(r"top(\d+)_label").astype(int)

# # Precompute counts per category/model/rank/label
# counts = (
#     long.groupby(["category", "model", "rank", "label"])
#         .size()
#         .reset_index(name="count")
# )

# def safe_save(fig, path):
#     try:
#         fig.savefig(path, dpi=160, bbox_inches="tight")
#         print(f"Saved: {path}")
#     finally:
#         plt.close(fig); gc.collect()

# def plot_model_category_stacked(model, category, top_k_per_rank=5, annotate=True):
#     sub = counts[(counts["model"] == model) & (counts["category"] == category)].copy()
#     if sub.empty:
#         print(f"No data for model={model}, category={category}")
#         return

#     # Build per-rank label counts and select top-k per rank; aggregate others as "Other"
#     ranks = [1, 2, 3, 4, 5]
#     per_rank = {}
#     all_labels = set()
#     for r in ranks:
#         sr = (sub[sub["rank"] == r]
#               .groupby("label")["count"].sum().sort_values(ascending=False))
#         if sr.empty:
#             per_rank[r] = {"Other": 0}
#             all_labels.add("Other")
#             continue
#         top = sr.head(top_k_per_rank)
#         other_sum = int(sr.iloc[top_k_per_rank:].sum()) if len(sr) > top_k_per_rank else 0
#         seg = {lab: int(c) for lab, c in top.items()}
#         if other_sum > 0:
#             seg["Other"] = other_sum
#         per_rank[r] = seg
#         for lab in seg.keys():
#             all_labels.add(lab)

#     # Order labels by total frequency across ranks (keep "Other" last)
#     totals = {lab: sum(per_rank[r].get(lab, 0) for r in ranks) for lab in all_labels}
#     labels_order = sorted([l for l in all_labels if l != "Other"], key=lambda x: -totals[x])
#     if "Other" in all_labels:
#         labels_order.append("Other")

#     # Prepare stacked data
#     x = np.arange(len(ranks))
#     width = 0.8
#     fig, ax = plt.subplots(figsize=(10, 5.2))
#     bottoms = np.zeros(len(ranks), dtype=int)

#     # Color palette
#     cmap = plt.cm.get_cmap("tab20", max(1, len(labels_order)))
#     colors = {lab: cmap(i % cmap.N) for i, lab in enumerate(labels_order)}

#     # Plot stacks
#     for lab in labels_order:
#         heights = np.array([per_rank[r].get(lab, 0) for r in ranks], dtype=int)
#         bars = ax.bar(x, heights, width=width, bottom=bottoms, color=colors[lab], edgecolor="white")
#         if annotate:
#             for xi, h, b in zip(x, heights, bottoms):
#                 if h >= 1:
#                     ax.text(xi, b + h / 2, f"{lab}:{h}", ha="center", va="center", fontsize=8, rotation=0)
#         bottoms += heights

#     ax.set_xticks(x)
#     ax.set_xticklabels([f"Top{r}" for r in ranks])
#     ax.set_ylabel("count")
#     ax.set_xlabel("prediction rank")
#     ax.set_title(f"{model} — {category.capitalize()} (label counts per rank)")
#     ax.yaxis.set_major_locator(MaxNLocator(integer=True))
#     ax.grid(axis="y", linestyle="--", alpha=0.3)

#     img_path = os.path.join(out_dir, f"{model}_{category}_top1-5_stacked_counts_{stamp}.png")
#     safe_save(fig, img_path)

#     # Save a compact CSV for this model/category
#     csv_path = os.path.join(out_dir, f"{model}_{category}_rank_label_counts_{stamp}.csv")
#     rows = []
#     for r in ranks:
#         for lab, c in sorted(per_rank[r].items(), key=lambda kv: -kv[1]):
#             rows.append({"model": model, "category": category, "rank": r, "label": lab, "count": c})
#     pd.DataFrame(rows).to_csv(csv_path, index=False)
#     print("Saved table:", csv_path)

# # Generate 6 plots (one per category for each model)
# for m in sorted(df["model"].unique()):
#     for cat in ["impact", "normal"]:
#         plot_model_category_stacked(m, cat, top_k_per_rank=5, annotate=True)

Saved: eval_outputs\slowfast_r50_8x8x1_impact_top1-5_stacked_counts_20251022_165416.png
Saved table: eval_outputs\slowfast_r50_8x8x1_impact_rank_label_counts_20251022_165416.csv
Saved: eval_outputs\slowfast_r50_8x8x1_normal_top1-5_stacked_counts_20251022_165416.png
Saved table: eval_outputs\slowfast_r50_8x8x1_normal_rank_label_counts_20251022_165416.csv
Saved: eval_outputs\tsm_r50_1x1x8_impact_top1-5_stacked_counts_20251022_165416.png
Saved table: eval_outputs\tsm_r50_1x1x8_impact_rank_label_counts_20251022_165416.csv
Saved: eval_outputs\tsm_r50_1x1x8_normal_top1-5_stacked_counts_20251022_165416.png
Saved table: eval_outputs\tsm_r50_1x1x8_normal_rank_label_counts_20251022_165416.csv
Saved: eval_outputs\tsn_r50_1x1x8_impact_top1-5_stacked_counts_20251022_165416.png
Saved table: eval_outputs\tsn_r50_1x1x8_impact_rank_label_counts_20251022_165416.csv
Saved: eval_outputs\tsn_r50_1x1x8_normal_top1-5_stacked_counts_20251022_165416.png
Saved table: eval_outputs\tsn_r50_1x1x8_normal_rank_label